In [1]:
import pandas as pd
import numpy as np
import datetime as dt 
from collections import Counter

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [42]:
casino = pd.read_csv('Online_casino_DIB.csv')
casino['ReqTimeUTC'] = pd.to_datetime(casino['ReqTimeUTC'])
casino = casino[casino['ReqTimeUTC'] <= '2020-02-29'].copy()
casino['TransactionType'] = casino['TransactionType'].map({'LOYALTYCARDDEBIT':'L2D', 'LOYALTYCARDCREDITCL':'L1D', 'LOYALTYCARDCREDIT':'L2W'})
casino = casino[(casino['TransactionType'] == 'L2D') & (casino['Status'] == 'APPROVED')].reset_index(drop=True)
casino = casino[['AccountIdentifier', 'ReqTimeUTC', 'TransactionAmount']]
casino = casino.sort_values(['AccountIdentifier', 'ReqTimeUTC']).reset_index(drop=True)
casino.rename(columns={'AccountIdentifier': 'customer', 'ReqTimeUTC':'timest', 'TransactionAmount': 'amount'}, inplace=True)
casino.head()

,customer,timest,amount
0,customer1,2019-03-01 17:24:43+00:00,30.0
1,customer1,2019-03-01 19:54:04+00:00,50.0
2,customer1,2019-03-01 22:44:15+00:00,40.0
3,customer1,2019-03-02 02:44:20+00:00,100.0
4,customer1,2019-03-02 18:28:39+00:00,25.0


### Problem framing
1. will a customer purchase at least once in the next three days?
2. is this a time series/ supervised / unsupervised learning problem?
3. what algorithms would be suitable for solving the problem?
4. what data do we need for solving this problem?

### Feature engineering

In [57]:
casino['date'] = casino['timest'].dt.normalize()
# date format must be in  timestamp as we must reindex the entire data frame.
# the previously used dt.date is not a timestamp and will not work :-()

In [44]:
casino.head()

,customer,timest,amount,date
0,customer1,2019-03-01 17:24:43+00:00,30.0,2019-03-01 00:00:00+00:00
1,customer1,2019-03-01 19:54:04+00:00,50.0,2019-03-01 00:00:00+00:00
2,customer1,2019-03-01 22:44:15+00:00,40.0,2019-03-01 00:00:00+00:00
3,customer1,2019-03-02 02:44:20+00:00,100.0,2019-03-02 00:00:00+00:00
4,customer1,2019-03-02 18:28:39+00:00,25.0,2019-03-02 00:00:00+00:00


In [45]:
daily_activity = casino.groupby(['customer', 'date']).agg(
    daily_sum = ('amount', 'sum'),
    tr_avg = ('amount', lambda x: x.sum() / x.count()),
    tr_count = ('timest', 'count')
    ).reset_index()
daily_activity.head()

,customer,date,daily_sum,tr_avg,tr_count
0,customer1,2019-03-01 00:00:00+00:00,120.0,40.000000,3
1,customer1,2019-03-02 00:00:00+00:00,125.0,62.500000,2
2,customer1,2019-03-03 00:00:00+00:00,200.0,100.000000,2
3,customer1,2019-03-04 00:00:00+00:00,160.0,53.333333,3
4,customer1,2019-03-05 00:00:00+00:00,240.0,80.000000,3


In [59]:
full_customer_df = pd.DataFrame()
for customer_id in daily_activity['customer'].unique():
    customer_df = daily_activity[daily_activity['customer'] == customer_id]
    full_range = list(pd.date_range(customer_df['date'].min(), customer_df['date'].max()))
    customer_df.set_index('date', inplace=True)
    customer_df = customer_df.reindex(full_range, fill_value=0)
    customer_df['customer'] = customer_id
    full_customer_df = pd.concat([full_customer_df, customer_df])
full_customer_df = full_customer_df.reset_index().copy()
full_customer_df


,date,customer,daily_sum,tr_avg,tr_count
0,2019-03-01 00:00:00+00:00,customer1,120.0,40.000000,3
1,2019-03-02 00:00:00+00:00,customer1,125.0,62.500000,2
2,2019-03-03 00:00:00+00:00,customer1,200.0,100.000000,2
3,2019-03-04 00:00:00+00:00,customer1,160.0,53.333333,3
4,2019-03-05 00:00:00+00:00,customer1,240.0,80.000000,3
...,...,...,...,...,...
164651,2019-09-12 00:00:00+00:00,customer996,0.0,0.000000,0
164652,2019-09-13 00:00:00+00:00,customer996,25.0,12.500000,2
164653,2019-09-22 00:00:00+00:00,customer997,50.0,50.000000,1
164654,2019-11-09 00:00:00+00:00,customer998,50.0,25.000000,2


In [61]:
customer_ids = full_customer_df['customer'].unique()
customer_ids

array(['customer1', 'customer10', 'customer100', ..., 'customer997',
       'customer998', 'customer999'], dtype=object)

In [102]:
ml_df = pd.DataFrame()
for customer in customer_ids:
    customer_df = full_customer_df[full_customer_df.customer==customer]
    customer_df = customer_df.reset_index(drop=True)

    features = []
    responses = []
    # Initial cutoff indices will be incremented
    x1, x14, y = 0, 14, 17

    while y<=len(customer_df):
        # Base
        feat_x1_x14 = customer_df['tr_count'][x1:x14].values.tolist()
        trans_next_3_days = np.count_nonzero(customer_df['tr_count'][x14:y])
        # response = [1 if trans_next_3_days!=0 else 0][0]
        response = [0 if trans_next_3_days!=0 else 1][0]
        responses.append(response)
        
        # Additional features
        x15 = np.count_nonzero(customer_df['tr_count'][x1:x14]) ## -- Nr of days with money deposits in the last 14 days
        x16 = np.count_nonzero(customer_df['tr_count'][x1+7:x14]) ## -- Nr of days with money deposits in the last 7 days
        x17 = np.count_nonzero(customer_df['tr_count'][x14-2:x14+1]) ## -- Nr of days with money deposits in the last 3 days
        x18 = customer_df['tr_count'][x1:x14].mean() # Average number of deposits in the last 14 days
        x19 = customer_df['tr_count'][x1:x14].max()  # Max number of daily deposits in the last 14 days
        x20 = customer_df['daily_sum'][x1+7:x14].sum()  # Amount deposited in the last 7 days
        x21 = customer_df['daily_sum'][x1+7:x14].mean() # Average Daily Amount deposited in the last 7 days
 
        feat_x1_x14.extend([x15, x16, x17, x18, x19, x20, x21, customer])
        features.append(feat_x1_x14)

        # increment
        x1+=1
        x14+=1
        y+=1
    df = pd.DataFrame(features)
    df["response"]=responses
    ml_df  = pd.concat([ml_df , df])
ml_df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,response
0,3.0,2.0,2.0,3.0,3.0,4.0,2.0,2.0,3.0,0.0,...,0.0,9.0,2.0,1.0,1.714286,4.0,400.0,57.142857,customer1,0.0
1,2.0,2.0,3.0,3.0,4.0,2.0,2.0,3.0,0.0,0.0,...,3.0,9.0,2.0,2.0,1.714286,4.0,285.0,40.714286,customer1,0.0
2,2.0,3.0,3.0,4.0,2.0,2.0,3.0,0.0,0.0,0.0,...,1.0,9.0,2.0,3.0,1.642857,4.0,110.0,15.714286,customer1,0.0
3,3.0,3.0,4.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,...,2.0,9.0,3.0,2.0,1.642857,4.0,145.0,20.714286,customer1,1.0
4,3.0,4.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,3.0,1.0,1.428571,4.0,145.0,20.714286,customer1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,customer996,1.0
164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,customer996,1.0
165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,customer996,1.0
166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,customer996,1.0


In [103]:
ml_df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,response
0,3.0,2.0,2.0,3.0,3.0,4.0,2.0,2.0,3.0,0.0,...,0.0,9.0,2.0,1.0,1.714286,4.0,400.0,57.142857,customer1,0.0
1,2.0,2.0,3.0,3.0,4.0,2.0,2.0,3.0,0.0,0.0,...,3.0,9.0,2.0,2.0,1.714286,4.0,285.0,40.714286,customer1,0.0
2,2.0,3.0,3.0,4.0,2.0,2.0,3.0,0.0,0.0,0.0,...,1.0,9.0,2.0,3.0,1.642857,4.0,110.0,15.714286,customer1,0.0
3,3.0,3.0,4.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,...,2.0,9.0,3.0,2.0,1.642857,4.0,145.0,20.714286,customer1,1.0
4,3.0,4.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,3.0,1.0,1.428571,4.0,145.0,20.714286,customer1,1.0
